# Projected 2021 Stats

In [1]:
#import dependencies
import pandas as pd
import requests
import time
import json
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func,inspect

# Website for 2021 Strength of Schedule
https://fftoolbox.fulltimefantasy.com/football/strength_of_schedule.cfm?type=i

In [2]:
query_url = f"https://api.sportsdata.io/v3/nfl/projections/json/PlayerGameProjectionStatsByWeek/2021REG/1?key=4339cb9aa27441699f1883848f498899"
response = requests.get(query_url).json()
print(json.dumps(response[0],indent=1))

{
 "GameKey": "202110116",
 "PlayerID": 18890,
 "SeasonType": 1,
 "Season": 2021,
 "GameDate": "2021-09-12T16:25:00",
 "Week": 1,
 "Team": "KC",
 "Opponent": "CLE",
 "HomeOrAway": "HOME",
 "Number": 15,
 "Name": "Patrick Mahomes",
 "Position": "QB",
 "PositionCategory": "OFF",
 "Activated": 1,
 "Played": 1,
 "Started": 1,
 "PassingAttempts": 20.0,
 "PassingCompletions": 13.3,
 "PassingYards": 161.9,
 "PassingCompletionPercentage": 34.5,
 "PassingYardsPerAttempt": 2.2,
 "PassingYardsPerCompletion": 3.4,
 "PassingTouchdowns": 0.7,
 "PassingInterceptions": 0.1,
 "PassingRating": 56.8,
 "PassingLong": 0.0,
 "PassingSacks": 0.0,
 "PassingSackYards": 0.0,
 "RushingAttempts": 1.0,
 "RushingYards": 5.1,
 "RushingYardsPerAttempt": 1.2,
 "RushingTouchdowns": 0.0,
 "RushingLong": 0.0,
 "ReceivingTargets": 0.0,
 "Receptions": 0.0,
 "ReceivingYards": 0.0,
 "ReceivingYardsPerReception": 0.0,
 "ReceivingTouchdowns": 0.0,
 "ReceivingLong": 0.0,
 "Fumbles": 0.0,
 "FumblesLost": 0.0,
 "PuntReturns": 0.0

In [3]:
wk1_2021_df = pd.DataFrame(columns=["PlayerID", "Name","Team", "Position", "Activated", "Played", "Started", "FantasyPoints", "FantasyPointsPPR","Opponent", "OpponentRank", "OpponentPositionRank"])
wk1_2021_df

,PlayerID,Name,Team,Position,Activated,Played,Started,FantasyPoints,FantasyPointsPPR,Opponent,OpponentRank,OpponentPositionRank


In [4]:
for i in range(0,len(response)):
    currentItem = response[i]
    wk1_2021_df.loc[i] = [response[i]["PlayerID"], response[i]["Name"], response[i]["Team"],response[i]["Position"], response[i]["Activated"], response[i]["Played"], response[i]["Started"], response[i]["FantasyPoints"], response[i]["FantasyPointsPPR"], response[i]["Opponent"], response[i]["OpponentRank"], response[i]["OpponentPositionRank"]]
wk1_2021_df.head()

,PlayerID,Name,Team,Position,Activated,Played,Started,FantasyPoints,FantasyPointsPPR,Opponent,OpponentRank,OpponentPositionRank
0,18890,Patrick Mahomes,KC,QB,1,1,1,12.2,12.2,CLE,None,None
1,19801,Josh Allen,BUF,QB,1,1,1,11.7,11.7,PIT,None,None
2,20889,Kyler Murray,ARI,QB,1,1,1,11.2,11.2,TEN,None,None
3,19781,Lamar Jackson,BAL,QB,1,1,1,10.8,10.8,LV,None,None
4,18055,Dak Prescott,DAL,QB,1,1,1,10.7,10.7,TB,None,None


In [5]:
print(wk1_2021_df["Team"].unique())

['KC' 'BUF' 'ARI' 'BAL' 'DAL' 'GB' 'SEA' 'LAC' 'TB' 'PHI' 'TEN' 'LAR'
 'CIN' 'JAX' 'ATL' 'CAR' 'CLE' 'MIN' 'MIA' 'LV' 'WAS' 'NYG' 'PIT' 'NYJ'
 'SF' 'DET' 'CHI' 'IND' 'NO' 'HOU' 'DEN' 'NE']


In [6]:
print(wk1_2021_df["Position"].unique())

['QB' 'RB' 'WR' 'TE' 'K']


In [7]:
print(len(wk1_2021_df))

948


In [31]:
STSchedule2021 = pd.read_csv('2021StrengthSchedule.csv')
STSchedule2021.head()

,TEAM,QB,RB,WR,TE,K,DEF,DL,LB,DB,PR,KR
0,NYJ,1,17,5,10,7,9,16,8,4,24,30
1,MIN,2,10,1,19,18,16,17,20,20,28,2
2,TEN,3,9,4,13,12,5,30,19,15,21,8
3,BAL,4,2,14,15,13,18,19,28,1,30,4
4,PHI,5,16,12,16,15,8,9,25,9,27,23


In [32]:
STSchedule2021.loc[(STSchedule2021.TEAM == 'LVR'),'TEAM']='LV'
STSchedule2021.head()

,TEAM,QB,RB,WR,TE,K,DEF,DL,LB,DB,PR,KR
0,NYJ,1,17,5,10,7,9,16,8,4,24,30
1,MIN,2,10,1,19,18,16,17,20,20,28,2
2,TEN,3,9,4,13,12,5,30,19,15,21,8
3,BAL,4,2,14,15,13,18,19,28,1,30,4
4,PHI,5,16,12,16,15,8,9,25,9,27,23


# QB

In [10]:
QBSTS = STSchedule2021.filter(items=['TEAM', 'QB'])
QBSTS.head()

,TEAM,QB
0,NYJ,1
1,MIN,2
2,TEN,3
3,BAL,4
4,PHI,5


In [11]:
QB2021_df = wk1_2021_df[wk1_2021_df['Position'] == "QB"]
QB2021_df

,PlayerID,Name,Team,Position,Activated,Played,Started,FantasyPoints,FantasyPointsPPR,Opponent,OpponentRank,OpponentPositionRank
0,18890,Patrick Mahomes,KC,QB,1,1,1,12.2,12.2,CLE,None,None
1,19801,Josh Allen,BUF,QB,1,1,1,11.7,11.7,PIT,None,None
2,20889,Kyler Murray,ARI,QB,1,1,1,11.2,11.2,TEN,None,None
3,19781,Lamar Jackson,BAL,QB,1,1,1,10.8,10.8,LV,None,None
4,18055,Dak Prescott,DAL,QB,1,1,1,10.7,10.7,TB,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
825,21816,Anthony Gordon,KC,QB,1,1,0,0.0,0.0,CLE,None,None
826,21819,Bryce Perkins,LAR,QB,1,1,0,0.0,0.0,CHI,None,None
843,22146,Ben DiNucci,DAL,QB,1,1,1,0.0,0.0,TB,None,None
844,22160,Reid Sinnett,MIA,QB,0,0,0,0.0,0.0,NE,None,None


In [12]:
QBSTSMerge = pd.merge(left=QB2021_df, right=QBSTS, how='left', left_on='Team', right_on='TEAM')
QBSTSMerge

,PlayerID,Name,Team,Position,Activated,Played,Started,FantasyPoints,FantasyPointsPPR,Opponent,OpponentRank,OpponentPositionRank,TEAM,QB
0,18890,Patrick Mahomes,KC,QB,1,1,1,12.2,12.2,CLE,None,None,KC,16
1,19801,Josh Allen,BUF,QB,1,1,1,11.7,11.7,PIT,None,None,BUF,29
2,20889,Kyler Murray,ARI,QB,1,1,1,11.2,11.2,TEN,None,None,ARI,17
3,19781,Lamar Jackson,BAL,QB,1,1,1,10.8,10.8,LV,None,None,BAL,4
4,18055,Dak Prescott,DAL,QB,1,1,1,10.7,10.7,TB,None,None,DAL,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,21816,Anthony Gordon,KC,QB,1,1,0,0.0,0.0,CLE,None,None,KC,16
112,21819,Bryce Perkins,LAR,QB,1,1,0,0.0,0.0,CHI,None,None,LAR,22
113,22146,Ben DiNucci,DAL,QB,1,1,1,0.0,0.0,TB,None,None,DAL,14
114,22160,Reid Sinnett,MIA,QB,0,0,0,0.0,0.0,NE,None,None,MIA,25


In [13]:
dpdqbmerge = QBSTSMerge.drop(columns=['TEAM','OpponentPositionRank'],axis=1)
QBmerge2 = dpdqbmerge.rename(columns={"QB": "OpponentPositionRank"})
QBmerge2

,PlayerID,Name,Team,Position,Activated,Played,Started,FantasyPoints,FantasyPointsPPR,Opponent,OpponentRank,OpponentPositionRank
0,18890,Patrick Mahomes,KC,QB,1,1,1,12.2,12.2,CLE,None,16
1,19801,Josh Allen,BUF,QB,1,1,1,11.7,11.7,PIT,None,29
2,20889,Kyler Murray,ARI,QB,1,1,1,11.2,11.2,TEN,None,17
3,19781,Lamar Jackson,BAL,QB,1,1,1,10.8,10.8,LV,None,4
4,18055,Dak Prescott,DAL,QB,1,1,1,10.7,10.7,TB,None,14
...,...,...,...,...,...,...,...,...,...,...,...,...
111,21816,Anthony Gordon,KC,QB,1,1,0,0.0,0.0,CLE,None,16
112,21819,Bryce Perkins,LAR,QB,1,1,0,0.0,0.0,CHI,None,22
113,22146,Ben DiNucci,DAL,QB,1,1,1,0.0,0.0,TB,None,14
114,22160,Reid Sinnett,MIA,QB,0,0,0,0.0,0.0,NE,None,25


# RB

In [14]:
RB2021_df = wk1_2021_df[wk1_2021_df['Position'] == "RB"]
RB2021_df.head()

,PlayerID,Name,Team,Position,Activated,Played,Started,FantasyPoints,FantasyPointsPPR,Opponent,OpponentRank,OpponentPositionRank
15,18877,Christian McCaffrey,CAR,RB,1,1,1,4.5,11.3,NYJ,None,None
21,18872,Dalvin Cook,MIN,RB,1,1,1,4.3,5.2,CIN,12,13
22,17959,Derrick Henry,TEN,RB,1,1,1,4.3,4.7,ARI,None,None
30,19798,Nick Chubb,CLE,RB,1,1,1,3.8,4.3,KC,None,None
31,17923,Ezekiel Elliott,DAL,RB,1,1,1,3.6,4.5,TB,None,None


In [15]:
RBSTS = STSchedule2021.filter(items=['TEAM', 'RB'])
RBSTS.head()

,TEAM,RB
0,NYJ,17
1,MIN,10
2,TEN,9
3,BAL,2
4,PHI,16


In [16]:
RBSTSMerge = pd.merge(left=RB2021_df, right=RBSTS, how='left', left_on='Team', right_on='TEAM')
RBSTSMerge

,PlayerID,Name,Team,Position,Activated,Played,Started,FantasyPoints,FantasyPointsPPR,Opponent,OpponentRank,OpponentPositionRank,TEAM,RB
0,18877,Christian McCaffrey,CAR,RB,1,1,1,4.5,11.3,NYJ,None,None,CAR,12
1,18872,Dalvin Cook,MIN,RB,1,1,1,4.3,5.2,CIN,12,13,MIN,10
2,17959,Derrick Henry,TEN,RB,1,1,1,4.3,4.7,ARI,None,None,TEN,9
3,19798,Nick Chubb,CLE,RB,1,1,1,3.8,4.3,KC,None,None,CLE,1
4,17923,Ezekiel Elliott,DAL,RB,1,1,1,3.6,4.5,TB,None,None,DAL,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,22736,Tory Carter,TEN,RB,0,0,0,0.0,0.0,ARI,None,None,TEN,9
209,22955,B.J. Emmons,LV,RB,1,1,0,0.0,0.0,BAL,None,None,LV,30
210,23002,Nick Ralston,DAL,RB,0,0,0,0.0,0.0,TB,None,None,DAL,20
211,23004,Adam Prentice,DEN,RB,0,0,0,0.0,0.0,NYG,None,None,DEN,11


In [17]:
dpdRBmerge = RBSTSMerge.drop(columns=['TEAM','OpponentPositionRank'],axis=1)
RBmerge2 = dpdRBmerge.rename(columns={"RB": "OpponentPositionRank"})
RBmerge2

,PlayerID,Name,Team,Position,Activated,Played,Started,FantasyPoints,FantasyPointsPPR,Opponent,OpponentRank,OpponentPositionRank
0,18877,Christian McCaffrey,CAR,RB,1,1,1,4.5,11.3,NYJ,None,12
1,18872,Dalvin Cook,MIN,RB,1,1,1,4.3,5.2,CIN,12,10
2,17959,Derrick Henry,TEN,RB,1,1,1,4.3,4.7,ARI,None,9
3,19798,Nick Chubb,CLE,RB,1,1,1,3.8,4.3,KC,None,1
4,17923,Ezekiel Elliott,DAL,RB,1,1,1,3.6,4.5,TB,None,20
...,...,...,...,...,...,...,...,...,...,...,...,...
208,22736,Tory Carter,TEN,RB,0,0,0,0.0,0.0,ARI,None,9
209,22955,B.J. Emmons,LV,RB,1,1,0,0.0,0.0,BAL,None,30
210,23002,Nick Ralston,DAL,RB,0,0,0,0.0,0.0,TB,None,20
211,23004,Adam Prentice,DEN,RB,0,0,0,0.0,0.0,NYG,None,11


# WR

In [18]:
WR2021_df = wk1_2021_df[wk1_2021_df['Position'] == "WR"]
WR2021_df.head()

,PlayerID,Name,Team,Position,Activated,Played,Started,FantasyPoints,FantasyPointsPPR,Opponent,OpponentRank,OpponentPositionRank
34,16470,Davante Adams,GB,WR,1,1,1,3.6,10.3,NO,None,None
37,18082,Tyreek Hill,KC,WR,1,1,1,3.5,5.0,CLE,None,None
43,20875,DK Metcalf,SEA,WR,1,1,1,3.1,4.6,IND,2,5
48,21042,A.J. Brown,TEN,WR,1,1,1,3.0,4.4,ARI,None,None
51,19802,Calvin Ridley,ATL,WR,1,1,1,3.0,4.6,PHI,None,None


In [19]:
WRSTS = STSchedule2021.filter(items=['TEAM', 'WR'])
WRSTS.head()

,TEAM,WR
0,NYJ,5
1,MIN,1
2,TEN,4
3,BAL,14
4,PHI,12


In [20]:
WRSTSMerge = pd.merge(left=WR2021_df, right=WRSTS, how='left', left_on='Team', right_on='TEAM')
WRSTSMerge

,PlayerID,Name,Team,Position,Activated,Played,Started,FantasyPoints,FantasyPointsPPR,Opponent,OpponentRank,OpponentPositionRank,TEAM,WR
0,16470,Davante Adams,GB,WR,1,1,1,3.6,10.3,NO,None,None,GB,20
1,18082,Tyreek Hill,KC,WR,1,1,1,3.5,5.0,CLE,None,None,KC,23
2,20875,DK Metcalf,SEA,WR,1,1,1,3.1,4.6,IND,2,5,SEA,15
3,21042,A.J. Brown,TEN,WR,1,1,1,3.0,4.4,ARI,None,None,TEN,4
4,19802,Calvin Ridley,ATL,WR,1,1,1,3.0,4.6,PHI,None,None,ATL,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,22991,Landen Akers,LAR,WR,0,0,0,0.0,0.0,CHI,None,None,LAR,8
371,23013,Damon Hazelton,GB,WR,0,0,0,0.0,0.0,NO,None,None,GB,20
372,23090,C.J. Saunders,CAR,WR,0,0,0,0.0,0.0,NYJ,None,None,CAR,3
373,23096,Travis Toivonen,SEA,WR,0,0,0,0.0,0.0,IND,None,None,SEA,15


In [21]:
dpdWRmerge = WRSTSMerge.drop(columns=['TEAM','OpponentPositionRank'],axis=1)
WRmerge2 = dpdWRmerge.rename(columns={"WR": "OpponentPositionRank"})
WRmerge2

,PlayerID,Name,Team,Position,Activated,Played,Started,FantasyPoints,FantasyPointsPPR,Opponent,OpponentRank,OpponentPositionRank
0,16470,Davante Adams,GB,WR,1,1,1,3.6,10.3,NO,None,20
1,18082,Tyreek Hill,KC,WR,1,1,1,3.5,5.0,CLE,None,23
2,20875,DK Metcalf,SEA,WR,1,1,1,3.1,4.6,IND,2,15
3,21042,A.J. Brown,TEN,WR,1,1,1,3.0,4.4,ARI,None,4
4,19802,Calvin Ridley,ATL,WR,1,1,1,3.0,4.6,PHI,None,18
...,...,...,...,...,...,...,...,...,...,...,...,...
370,22991,Landen Akers,LAR,WR,0,0,0,0.0,0.0,CHI,None,8
371,23013,Damon Hazelton,GB,WR,0,0,0,0.0,0.0,NO,None,20
372,23090,C.J. Saunders,CAR,WR,0,0,0,0.0,0.0,NYJ,None,3
373,23096,Travis Toivonen,SEA,WR,0,0,0,0.0,0.0,IND,None,15


# TE

In [22]:
TE2021_df = wk1_2021_df[wk1_2021_df['Position'] == "TE"]
TE2021_df.head()

,PlayerID,Name,Team,Position,Activated,Played,Started,FantasyPoints,FantasyPointsPPR,Opponent,OpponentRank,OpponentPositionRank
44,15048,Travis Kelce,KC,TE,1,1,1,3.1,4.8,CLE,None,None
64,16964,Darren Waller,LV,TE,1,1,1,2.5,4.1,BAL,None,None
72,19063,George Kittle,SF,TE,1,1,1,2.4,3.8,DET,None,None
108,19803,Mark Andrews,BAL,TE,1,1,1,2.1,3.2,LV,None,None
121,22508,Kyle Pitts,ATL,TE,1,1,1,2.0,3.2,PHI,None,None


In [23]:
TESTS = STSchedule2021.filter(items=['TEAM', 'TE'])
TESTS.head()

,TEAM,TE
0,NYJ,10
1,MIN,19
2,TEN,13
3,BAL,15
4,PHI,16


In [24]:
TESTSMerge = pd.merge(left=TE2021_df, right=TESTS, how='left', left_on='Team', right_on='TEAM')
TESTSMerge

,PlayerID,Name,Team,Position,Activated,Played,Started,FantasyPoints,FantasyPointsPPR,Opponent,OpponentRank,OpponentPositionRank,TEAM,TE
0,15048,Travis Kelce,KC,TE,1,1,1,3.1,4.8,CLE,None,None,KC,4
1,16964,Darren Waller,LV,TE,1,1,1,2.5,4.1,BAL,None,None,LV,21
2,19063,George Kittle,SF,TE,1,1,1,2.4,3.8,DET,None,None,SF,27
3,19803,Mark Andrews,BAL,TE,1,1,1,2.1,3.2,LV,None,None,BAL,15
4,22508,Kyle Pitts,ATL,TE,1,1,1,2.0,3.2,PHI,None,None,ATL,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,23060,Jake Hausmann,NYG,TE,0,0,0,0.0,0.0,DEN,None,None,NYG,20
195,23063,Bernhard Seikovits,ARI,TE,0,0,0,0.0,0.0,TEN,None,None,ARI,30
196,23084,Shane Zylstra,MIN,TE,0,0,0,0.0,0.0,CIN,12,5,MIN,19
197,23101,Parker Hesse,ATL,TE,0,0,0,0.0,0.0,PHI,None,None,ATL,3


In [25]:
dpdTEmerge = TESTSMerge.drop(columns=['TEAM','OpponentPositionRank'],axis=1)
TEmerge2 = dpdTEmerge.rename(columns={"TE": "OpponentPositionRank"})
TEmerge2

,PlayerID,Name,Team,Position,Activated,Played,Started,FantasyPoints,FantasyPointsPPR,Opponent,OpponentRank,OpponentPositionRank
0,15048,Travis Kelce,KC,TE,1,1,1,3.1,4.8,CLE,None,4
1,16964,Darren Waller,LV,TE,1,1,1,2.5,4.1,BAL,None,21
2,19063,George Kittle,SF,TE,1,1,1,2.4,3.8,DET,None,27
3,19803,Mark Andrews,BAL,TE,1,1,1,2.1,3.2,LV,None,15
4,22508,Kyle Pitts,ATL,TE,1,1,1,2.0,3.2,PHI,None,3
...,...,...,...,...,...,...,...,...,...,...,...,...
194,23060,Jake Hausmann,NYG,TE,0,0,0,0.0,0.0,DEN,None,20
195,23063,Bernhard Seikovits,ARI,TE,0,0,0,0.0,0.0,TEN,None,30
196,23084,Shane Zylstra,MIN,TE,0,0,0,0.0,0.0,CIN,12,19
197,23101,Parker Hesse,ATL,TE,0,0,0,0.0,0.0,PHI,None,3


# K

In [26]:
K2021_df = wk1_2021_df[wk1_2021_df['Position'] == "K"]
K2021_df.head()

,PlayerID,Name,Team,Position,Activated,Played,Started,FantasyPoints,FantasyPointsPPR,Opponent,OpponentRank,OpponentPositionRank
70,14688,Justin Tucker,BAL,K,1,1,1,2.4,2.4,LV,None,None
73,19073,Harrison Butker,KC,K,1,1,1,2.4,2.4,CLE,None,None
80,8750,Ryan Succop,TB,K,1,1,1,2.3,2.3,DAL,None,None
83,22108,Tyler Bass,BUF,K,1,1,1,2.3,2.3,PIT,None,None
84,1410,Robbie Gould,SF,K,1,1,1,2.2,2.2,DET,None,None


In [27]:
KSTS = STSchedule2021.filter(items=['TEAM', 'K'])
KSTS.head()

,TEAM,K
0,NYJ,7
1,MIN,18
2,TEN,12
3,BAL,13
4,PHI,15


In [28]:
KSTSMerge = pd.merge(left=K2021_df, right=KSTS, how='left', left_on='Team', right_on='TEAM')
KSTSMerge.head()

,PlayerID,Name,Team,Position,Activated,Played,Started,FantasyPoints,FantasyPointsPPR,Opponent,OpponentRank,OpponentPositionRank,TEAM,K
0,14688,Justin Tucker,BAL,K,1,1,1,2.4,2.4,LV,None,None,BAL,13
1,19073,Harrison Butker,KC,K,1,1,1,2.4,2.4,CLE,None,None,KC,28
2,8750,Ryan Succop,TB,K,1,1,1,2.3,2.3,DAL,None,None,TB,31
3,22108,Tyler Bass,BUF,K,1,1,1,2.3,2.3,PIT,None,None,BUF,29
4,1410,Robbie Gould,SF,K,1,1,1,2.2,2.2,DET,None,None,SF,24


In [29]:
dpdKmerge = KSTSMerge.drop(columns=['TEAM','OpponentPositionRank'],axis=1)
Kmerge2 = dpdKmerge.rename(columns={"K": "OpponentPositionRank"})
Kmerge2.head()

,PlayerID,Name,Team,Position,Activated,Played,Started,FantasyPoints,FantasyPointsPPR,Opponent,OpponentRank,OpponentPositionRank
0,14688,Justin Tucker,BAL,K,1,1,1,2.4,2.4,LV,None,13
1,19073,Harrison Butker,KC,K,1,1,1,2.4,2.4,CLE,None,28
2,8750,Ryan Succop,TB,K,1,1,1,2.3,2.3,DAL,None,31
3,22108,Tyler Bass,BUF,K,1,1,1,2.3,2.3,PIT,None,29
4,1410,Robbie Gould,SF,K,1,1,1,2.2,2.2,DET,None,24


# Combine 2021 Position Data

In [30]:
Proj2021df = pd.concat([QBmerge2,RBmerge2,TEmerge2,WRmerge2,Kmerge2])
Proj2021df

,PlayerID,Name,Team,Position,Activated,Played,Started,FantasyPoints,FantasyPointsPPR,Opponent,OpponentRank,OpponentPositionRank
0,18890,Patrick Mahomes,KC,QB,1,1,1,12.2,12.2,CLE,None,16
1,19801,Josh Allen,BUF,QB,1,1,1,11.7,11.7,PIT,None,29
2,20889,Kyler Murray,ARI,QB,1,1,1,11.2,11.2,TEN,None,17
3,19781,Lamar Jackson,BAL,QB,1,1,1,10.8,10.8,LV,None,4
4,18055,Dak Prescott,DAL,QB,1,1,1,10.7,10.7,TB,None,14
...,...,...,...,...,...,...,...,...,...,...,...,...
40,22115,J.J. Molson,GB,K,0,0,0,0.0,0.0,NO,None,21
41,22122,Lirim Hajrullahu,DAL,K,0,0,0,0.0,0.0,TB,None,4
42,22863,Riley Patterson,MIN,K,1,1,1,0.0,0.0,CIN,None,18
43,23086,Brian Johnson,CHI,K,0,0,0,0.0,0.0,LAR,None,10


In [33]:
Proj2021df.to_csv ('Projected2021.csv', index = False, header=True)

# Groupby stats